In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
def extractdepCampusNote(strDepartment):
    dep, campus, note, isNote = '', '0', '0', True
    if(strDepartment.find('校區') > -1):
        campus = ''
        isNote = False
    for i in range(len(strDepartment)):
        if(strDepartment[i] == '(' or strDepartment[i] == '（'):
            if(isNote):
                note = ""
            for j in range(i+1, len(strDepartment)):
                if(strDepartment[j] == ')' or strDepartment[j] == '）'):
                    break
                if(isNote):
                    note += strDepartment[j]
                else:
                    campus += strDepartment[j]
            break
        elif(strDepartment[i] == ' ' or strDepartment[i] == '　'):
            continue
        dep += strDepartment[i]
    return list((dep, campus, note))

def validSchool(campusList, addressList):
    if(len(campusList) != len(addressList)):
        return False
    if(len(campusList) == 1):
        if(campusList[0] == '0' and addressList[0] != '0'):
            return True
        else:
            return False
    else:
        for i in range(len(campusList)):
            if(campusList[i] == '0' or addressList[i] == '0'):
                return False
        return True

### 建立112甄選db資料

In [38]:
dfSchool = pd.read_excel("./文件集/學校統整v4.xlsx", dtype=str)
dfRec = pd.read_excel('./文件集/112四技二專甄選_一般組v2.xlsx', dtype=str)
dfRec

,學校ID,校名,科系,備註,校區,類群ID,類群名稱
0,101,國立臺灣科技大學,企業管理系,0,0,09,商業與管理群
1,101,國立臺灣科技大學,化學工程系,0,0,05,化工群
2,101,國立臺灣科技大學,建築系,0,0,06,土木與建築群
3,101,國立臺灣科技大學,建築系,0,0,07,設計群
4,101,國立臺灣科技大學,應用外語系,0,0,15,外語群英語類
...,...,...,...,...,...,...,...
2743,831,明道學校財團法人明道大學,景觀與環境設計學系,0,0,14,農業群
2744,831,明道學校財團法人明道大學,智慧暨精緻農業學系,0,0,02,動力機械群
2745,831,明道學校財團法人明道大學,智慧暨精緻農業學系,0,0,14,農業群
2746,831,明道學校財團法人明道大學,智慧能源工程學系,0,0,02,動力機械群


In [39]:
validSchoolIds, invalidSchools = set(), []
for schoolID in dfRec['學校ID'].unique():
    mask = dfSchool['學校ID'] == schoolID
    campusList, addressList = dfSchool[mask]['校區'].unique(), dfSchool[mask]['地址'].unique()
    if(validSchool(campusList, addressList)):
        validSchoolIds.add(schoolID)
    else:
        invalidSchools.append(schoolID)
for id in invalidSchools:
    mask = dfRec['學校ID'] == id
    print(dfRec[mask]['校名'].iloc[0])

國立高雄科技大學
國立臺北護理健康大學
崇右影藝科技大學
國立臺東專科學校
國立陽明交通大學
國立高雄師範大學
國立彰化師範大學
國立臺中教育大學
國立屏東大學
臺北醫學大學
中國醫藥大學


In [40]:
data = []
db = mysql.connector.connect(user='root', password='root', host='localhost', database='四技二專')
cursor = db.cursor()
sql1 = "select 分校ID from 分校 where 學校ID = %s and 校區 = %s"
sql2 = 'select 科系ID from 科系 where 科系名稱 = %s'

for i in range(len(dfRec)):
    schoolID = dfRec['學校ID'].iloc[i]
    classID = dfRec['類群ID'].iloc[i]
    if(schoolID in validSchoolIds):
        dep, campus = dfRec['科系'].iloc[i], dfRec['校區'].iloc[i]
        cursor.execute(sql1, (schoolID, campus))
        campusID = cursor.fetchall()[0][0]
        cursor.execute(sql2, (dep,))
        depID = cursor.fetchall()[0][0]
        data.append((classID, campusID,depID))

In [46]:
sql = 'insert into 112甄選 values(%s, %s, %s)'
cursor.executemany(sql, data)
db.commit()

In [47]:
cursor.close()
db.close()
len(data)

2530

### 彙整112四技二專甄選_一般組excel

In [24]:
dfSchool = pd.read_excel('./文件集/學校統整v4.xlsx')
dfRec = pd.read_excel('./文件集/112四技二專甄選_一般組.xls')
dfSchool = dfSchool.astype('str')
dfRec = dfRec.astype('str')
dfRec = dfRec[['學校代碼', '校名', '系名', '招生群（類）別']]
dfRec.columns = ['學校ID', '校名', '科系', '類群名稱']
dfRec

,學校ID,校名,科系,類群名稱
0,101,國立臺灣科技大學,機械工程系,01 機械群
1,101,國立臺灣科技大學,機械工程系,02 動力機械群
2,101,國立臺灣科技大學,材料科學與工程系,01 機械群
3,101,國立臺灣科技大學,材料科學與工程系,03 電機與電子群電機類
4,101,國立臺灣科技大學,材料科學與工程系,05 化工群
...,...,...,...,...
2743,831,明道學校財團法人明道大學,智慧暨精緻農業學系,02 動力機械群
2744,831,明道學校財團法人明道大學,智慧暨精緻農業學系,14 農業群
2745,831,明道學校財團法人明道大學,休閒保健學系,17 餐旅群
2746,831,明道學校財團法人明道大學,數位設計學系,07 設計群


In [25]:
keys = ['科系', '校區', '備註', '類群ID', '類群名稱']
dicData = {'科系':[], '校區':[], '備註':[], '類群名稱':[], '類群ID':[]} 
for i in range(len(dfRec)):
   tempList = extractdepCampusNote(dfRec.iloc[i]['科系'])
   tempList.extend(dfRec.iloc[i]['類群名稱'].split())
   for j in range(len(tempList)):
      dicData[keys[j]].append(tempList[j])

for key in dicData.keys():
   dfRec[key] = dicData[key]
dfRec = dfRec.iloc[:, [0, 1, 2, 5, 4, 6, 3]].sort_values(by=['學校ID', '校區', '科系'], ignore_index=True)
dfRec

,學校ID,校名,科系,備註,校區,類群ID,類群名稱
0,101,國立臺灣科技大學,企業管理系,0,0,09,商業與管理群
1,101,國立臺灣科技大學,化學工程系,0,0,05,化工群
2,101,國立臺灣科技大學,建築系,0,0,06,土木與建築群
3,101,國立臺灣科技大學,建築系,0,0,07,設計群
4,101,國立臺灣科技大學,應用外語系,0,0,15,外語群英語類
...,...,...,...,...,...,...,...
2743,831,明道學校財團法人明道大學,景觀與環境設計學系,0,0,14,農業群
2744,831,明道學校財團法人明道大學,智慧暨精緻農業學系,0,0,02,動力機械群
2745,831,明道學校財團法人明道大學,智慧暨精緻農業學系,0,0,14,農業群
2746,831,明道學校財團法人明道大學,智慧能源工程學系,0,0,02,動力機械群


In [26]:
dfRec = dfRec.astype('str')
dfRec.to_excel('./文件集/112四技二專甄選_一般組v2.xlsx', index=False)

### 建立甄選Data到DB

In [2]:
dfRegular = pd.read_excel("./文件集/112四技二專甄選_一般組.xls")
dfRegular['招生群（類）別']

0             01 機械群
1           02 動力機械群
2             01 機械群
3       03 電機與電子群電機類
4             05 化工群
            ...     
2743        02 動力機械群
2744          14 農業群
2745          17 餐旅群
2746          07 設計群
2747       20 藝術群影視類
Name: 招生群（類）別, Length: 2748, dtype: object

In [9]:
recom_adm_data = []
classes = np.sort(dfRegular['招生群（類）別'].unique())
for group in classes:
    recom_adm_data.append(tuple(group.split()))
recom_adm_data

[('01', '機械群'),
 ('02', '動力機械群'),
 ('03', '電機與電子群電機類'),
 ('04', '電機與電子群資電類'),
 ('05', '化工群'),
 ('06', '土木與建築群'),
 ('07', '設計群'),
 ('08', '工程與管理類'),
 ('09', '商業與管理群'),
 ('10', '衛生與護理類'),
 ('11', '食品群'),
 ('12', '家政群幼保類'),
 ('13', '家政群生活應用類'),
 ('14', '農業群'),
 ('15', '外語群英語類'),
 ('16', '外語群日語類'),
 ('17', '餐旅群'),
 ('18', '海事群'),
 ('19', '水產群'),
 ('20', '藝術群影視類'),
 ('21', '資管類')]

In [12]:
conn = mysql.connector.connect(user='root', password='root', host='localhost', database='四技二專')
cursor = conn.cursor()
sql = "insert into 甄選 values(%s, %s, null)"
cursor.executemany(sql, recom_adm_data)
conn.commit()

In [13]:
cursor.close()
conn.close()